In [ ]:
using Pkg
cd(@__DIR__)
cd("..")
Pkg.activate(".")

using MyDiffMLP
using MyDiffMLP.MyAD
using MyDiffMLP.MyNN

In [3]:
using JLD2
data = prepare_dataset(10000, 0.8)

X_train = data.X_train
y_train = data.y_train
X_test = data.X_test
y_test = data.y_test

    
nothing

UndefVarError: UndefVarError: `prepare_dataset` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [5]:
using Printf, Statistics, Random

function create_batches(X, Y; batchsize=64, shuffle=true)
    idxs = collect(1:size(X, 2))
    if shuffle
        Random.shuffle!(idxs)
    end
    return [(X[:, idxs[i:min(i+batchsize-1, end)]],
             Y[:, idxs[i:min(i+batchsize-1, end)]])
             for i in 1:batchsize:length(idxs)]
end

model = Chain(
    Dense(size(X_train, 1), 32, relu),
    Dense(32, 1, sigmoid)
)

function bce(ŷ, y)
    ϵ = 1e-7
    ŷ_clipped = clamp.(ŷ, ϵ, 1 .- ϵ)
    return -mean(y .* log.(ŷ_clipped) .+ (1 .- y) .* log.(1 .- ŷ_clipped))
end

function bce_grad(ŷ, y)
    ϵ = 1e-7
    return (ŷ .- y) ./ (clamp.(ŷ .* (1 .- ŷ), ϵ, 1.0)) ./ size(y, 2)
end

accuracy(ŷ, y) = mean((ŷ .> 0.5) .== (y .> 0.5))

epochs = 5
batchsize = 64
η = 0.001

params = parameters(model)
state = AdamState(params)

for epoch in 1:epochs
    total_loss = 0.0
    total_acc = 0.0
    num_batches = 0

    batches = create_batches(X_train, y_train; batchsize=batchsize)

    t = @elapsed begin
        for (x, y) in batches
            x_node = Variable(x, zeros(size(x)))
            y_node = Variable(y, zeros(size(y)))

            out = model(x_node)
            graph = topological_sort(out)
            forward!(graph)

            ŷ = out.output
            l = bce(ŷ, y)
            total_loss += l
            total_acc += accuracy(ŷ, y)
            num_batches += 1

            out.gradient = bce_grad(ŷ, y)

            zero_gradients!(model)
            backward!(graph, out.gradient)
            update_adam!(state, params, η)

        end
    end

     
    train_loss = total_loss / num_batches
    train_acc = total_acc / num_batches

    # --- Ewaluacja ---
    x_eval = Variable(X_test, zeros(size(X_test)))
    out_eval = model(x_eval)
    forward!(topological_sort(out_eval))
    test_pred = out_eval.output
    test_loss = bce(test_pred, y_test)
    test_acc = accuracy(test_pred, y_test)

    println(@sprintf("Epoch: %d (%.2fs) \tTrain: (l: %.2f, a: %.2f) \tTest: (l: %.2f, a: %.2f)",
        epoch, t, train_loss, train_acc, test_loss, test_acc))
end

Epoch: 1 (6.06s) 	Train: (l: 0.59, a: 0.81) 	Test: (l: 0.49, a: 0.86)
Epoch: 2 (5.42s) 	Train: (l: 0.36, a: 0.93) 	Test: (l: 0.39, a: 0.87)
Epoch: 3 (5.49s) 	Train: (l: 0.23, a: 0.96) 	Test: (l: 0.34, a: 0.87)
Epoch: 4 (5.62s) 	Train: (l: 0.16, a: 0.97) 	Test: (l: 0.32, a: 0.87)
Epoch: 5 (5.16s) 	Train: (l: 0.11, a: 0.99) 	Test: (l: 0.32, a: 0.87)
